<a href="https://colab.research.google.com/github/simulate111/Deep-Learning-in-Human-Language-Technology/blob/main/course_project_Reza_ChGT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning in Human Language Technology Project (Template)

- Student(s) Name(s): Mohammadreza Akhtari
- Date: October 2024
- Chosen Corpus:
- Contributions (if group project): -

### Corpus information

- Description of the chosen corpus:
- Paper(s) and other published materials related to the corpus:
- Random baseline performance and expected performance for recent machine learned models:

---

## 1. Setup

In [1]:
!pip install datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00


In [2]:
# Your code to install and import libraries etc. here
# Installing necessary libraries
!pip install datasets transformers scikit-learn

# Importing the libraries
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics import accuracy_score, mean_absolute_error
import random

# Setting a seed for reproducibility
random.seed(42)
torch.manual_seed(42)



---

## 2. Data download, sampling and preprocessing

### 2.1. Download the corpus

In [3]:
# Your code to download the corpus here
# Downloading the Amazon Reviews Multilingual dataset using the datasets library
dataset = load_dataset("mteb/amazon_reviews_multi", trust_remote_code=True)

# Filtering to get English data only
train_data = dataset['train'].filter(lambda x: x['id'].startswith('en'))
validation_data = dataset['validation'].filter(lambda x: x['id'].startswith('en'))
test_data = dataset['test'].filter(lambda x: x['id'].startswith('en'))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


amazon_reviews_multi.py:   0%|          | 0.00/6.17k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/61.4M [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/53.1M [00:00<?, ?B/s]

...

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/1200000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

### 2.2. Sampling and preprocessing

In [4]:
# Sampling the data if needed (example: taking a smaller subset)
sampled_train_data = train_data.shuffle(seed=42).select(range(20000))  # Adjust sample size if needed
sampled_validation_data = validation_data.shuffle(seed=42).select(range(5000))  # Adjust sample size if needed

# Preprocessing: Tokenizing the text
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenize the datasets
tokenized_train = sampled_train_data.map(preprocess_function, batched=True)
tokenized_validation = sampled_validation_data.map(preprocess_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

---

## 3. Machine learning model

### 3.1. Model training

In [5]:
# Your code to train the transformer based model on the training set and evaluate the performance on the validation set here
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, mean_absolute_error

# Define the number of labels
num_labels = 5  # Since you have star ratings from 1 to 5

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=num_labels)

# Prepare the datasets for training
train_encoded = tokenized_train.rename_column("label", "labels")
validation_encoded = tokenized_validation.rename_column("label", "labels")

# Set the format of the dataset for PyTorch
train_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
validation_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=3,               # total number of training epochs
    per_device_train_batch_size=16,   # batch size per device during training
    per_device_eval_batch_size=16,    # batch size for evaluation
    warmup_steps=500,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                 # strength of weight decay
    logging_dir='./logs',              # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",       # evaluate every epoch
)

# Define a compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    mae = mean_absolute_error(labels, predictions)
    return {"accuracy": accuracy, "mae": mae}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=validation_encoded,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Mae
1,1.028800,0.981429,0.582600,0.532000
2,0.862300,0.915534,0.612400,0.471600
3,0.650800,0.988351,0.614400,0.457200


TrainOutput(global_step=3750, training_loss=0.8775240439097086, metrics={'train_runtime': 6072.2414, 'train_samples_per_second': 9.881, 'train_steps_per_second': 0.618, 'total_flos': 1.578708854784e+16, 'train_loss': 0.8775240439097086, 'epoch': 3.0})

### 3.2 Hyperparameter optimization

In [ ]:
# Your code for hyperparameter optimization here
# Using grid search or similar for hyperparameter optimization (example shown here)
best_acc = 0
best_lr = 0

for lr in [1e-5, 3e-5]: # 2e-5,
    training_args.learning_rate = lr
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_encoded,
        eval_dataset=validation_encoded,
        compute_metrics=compute_metrics
    )
    trainer.train()
    metrics = trainer.evaluate()
    if metrics['eval_accuracy'] > best_acc:
        best_acc = metrics['eval_accuracy']
        best_lr = lr

print(f"Best Learning Rate: {best_lr}")


Epoch,Training Loss,Validation Loss,Accuracy,Mae
1,0.491700,1.259704,0.601600,0.464600
2,0.268800,1.537509,0.602800,0.471000
3,0.639400,1.773271,0.601200,0.476600


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Mae
1,0.274300,2.091313,0.601400,0.481800


### 3.3. Evaluation on test set

In [ ]:
# Your code to evaluate the final model on the test set here
# Evaluate final model on test set
trainer.args.learning_rate = best_lr
trainer.evaluate(test_encoded)


### 3.4. Cross-lingual experiments

In [ ]:
# Your code to train and evaluate the cross-lingual model
# Zero-shot cross-lingual transfer: evaluate the model trained on English on other languages
languages = ['de', 'es', 'fr', 'ja', 'zh']

cross_lingual_results = {}
for lang in languages:
    test_data_lang = dataset['test'][lang]


---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

### 4.2 Results

(Briefly summarize your results)

### 4.3 Relation to random baseline / expected performance / state of the art

(Compare your results with the random and state-of-the-art performance)

---

## 5. Bonus Task (optional)

### 5.1. and 5.2. Model and Data selection

(Briefly describe which model was used and why. Also, describe how the test data was downsampled, include relevant code.)

### 5.3. Prompt design

(Include your final prompt here. Also, explain here all prompt engineering insights you learned during the project.)

### 5.4. Generate

In [ ]:
# Your code to run the generative model and extract predictions from the model output.

### 5.5. Evaluation and results

(Briefly summarize your results)

### 5.6 Error analysis (group projects only)

(Present the error analysis results here)